<a href="https://www.kaggle.com/code/chhelp/unit-3-deep-q-learning-with-atari-games-using?scriptVersionId=143628561" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 安装 RL-Baseliness3 Zoo 及其依赖项

In [ ]:
# For now we install this update of RL-Baselines3 Zoo
!pip install git+https://github.com/DLR-RM/rl-baselines3-zoo@update/hf

In [ ]:
!apt-get install -y swig cmake ffmpeg

In [ ]:
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

* 创建虚拟屏幕

In [ ]:
%%capture
!apt install -y python-opengl
!apt install -y ffmpeg
!apt install -y xvfb
!pip3 install -y pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# 训练 Deep Q-learning Agent 来玩 Space Invaders

* 创建一个 超参数配置文件 dqn.yml
    * 使用 Atari Wrapper 来预处理输入(帧缩减、灰度、堆栈 4 帧)
    * 使用 CnnPolicy 卷积层来处理帧
    * n_timesteps 训练次数 1 百万
    * 内存(经验回放) 大小为 100000 保存用于再次训练代理的经验步数
* 在超参数优化方面 建议关注下述三个
    * learning_rate
    * buffer_size (Experience Memory size)
    * batch_size

In [ ]:
# 创建 yml 文件
import yaml

config = {
    'SpaceInvadersNoFrameskip-v4': {
        'env_wrapper': ['stable_baselines3.common.atari_wrappers.AtariWrapper'],
        'frame_stack': 4,
        'policy': 'CnnPolicy',
        'n_timesteps': 1e6, # 最初是 1e7 对我来说训练的时间太长了
        'buffer_size': 100000,
        'learning_rate': 1e-4,
        'batch_size': 32,
        'learning_starts': 100000,
        'target_update_interval': 1000,
        'train_freq': 4,
        'gradient_steps': 1,
        'exploration_fraction': 0.1,
        'exploration_final_eps': 0.01,
        'optimize_memory_usage': False
    }
}

with open('dqn.yml', 'w') as f:
    yaml.dump(config, f)



* 在 --algo 之后定义算法 在 -f 之后保存模型 超参数配置在 -c 之后

In [ ]:
!python -m rl_zoo3.train --algo dqn  --env SpaceInvadersNoFrameskip-v4 -f logs/ -c dqn.yml

# 评估一下 Agent

In [ ]:
!python -m rl_zoo3.enjoy  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps 5000  --folder logs/

# 在 hub 上发布经过训练的 Agent

In [ ]:
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
notebook_login()
!git config --global credential.helper store

# There are some trouble

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name Atari_Games  -orga Solitary12138  -f logs/